# BERT2BERT

На основе https://github.com/huggingface/notebooks/blob/master/examples/summarization.ipynb

In [1]:
!wget https://www.dropbox.com/s/mbj3sb6jaw3d9s3/judgements_test.json
!wget https://www.dropbox.com/s/hbh7ioaiet16fcw/judgements_train.json

--2021-06-07 21:28:33--  https://www.dropbox.com/s/mbj3sb6jaw3d9s3/judgements_test.json
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/mbj3sb6jaw3d9s3/judgements_test.json [following]
--2021-06-07 21:28:34--  https://www.dropbox.com/s/raw/mbj3sb6jaw3d9s3/judgements_test.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc03661a099c745e8ffa29207597.dl.dropboxusercontent.com/cd/0/inline/BP942pUwmCrDeItAG4ZikahDsgJxGfC07Aw-N6WGneb_vhVDB3zr_0at7hN3UhILk14v-Nk-gB96Wy6MwFKTaI9Z--VsjMtwrnixN3pbQ1YRJ7VfCQchmJLFx0BFl8piN3HnOahaeECyzcMPv5q_aqSe/file# [following]
--2021-06-07 21:28:34--  https://uc03661a099c745e8ffa29207597.dl.dropboxusercontent.com/cd/0/inline/BP942pUwmCrDeItAG4ZikahDsgJxGfC07Aw-N6WGneb_vhVDB3zr_0at7hN3UhILk1

In [2]:
! pip install transformers datasets sentencepiece

     |████████████████████████████████| 2.3MB 8.2MB/s 
     |████████████████████████████████| 235kB 49.9MB/s 
     |████████████████████████████████| 1.2MB 52.0MB/s 
     |████████████████████████████████| 901kB 50.2MB/s 
     |████████████████████████████████| 3.3MB 51.1MB/s 
     |████████████████████████████████| 245kB 44.5MB/s 
     |████████████████████████████████| 122kB 58.4MB/s 


In [1]:
import json

In [2]:
with open ('judgements_train.json', encoding='utf-8') as f:
  data_train = json.load(f)

with open ('judgements_test.json', encoding='utf-8') as f:
  data_test = json.load(f)

In [3]:
from sklearn.model_selection import train_test_split

_, val_keys = train_test_split(list(data_train.keys()), test_size=0.15, random_state=10)

data_val = {}

for k in val_keys:
  data_val[k] = data_train[k]
  data_train.pop(k)

In [4]:
from transformers import AutoTokenizer


model_name = "dmitry-vorobiev/rubert_ria_headlines"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
'''import nltk
nltk.download('punkt')
from nltk import sent_tokenize

def trunc(text):
  trunc_text = ''
  sentences = sent_tokenize(text)
  decision_sentence = sentences[-1]
  min_len = len(tokenizer(decision_sentence)['input_ids'])
  for sent in sentences:
    sent_len = len(tokenizer(sent)['input_ids'])-2
    if sent_len+min_len<512:
      trunc_text += sent
      min_len += sent_len
    else:
      break
  trunc_text += '[PAD]'*(512-min_len)
  trunc_text += decision_sentence
  return trunc_text'''

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
n_data_train = {'id': [], 'texts': [], 'summaries': []}

for ind in data_train.keys():
  n_data_train['id'].append(ind)
  n_data_train['texts'].append(data_train[ind]['text'])
  n_data_train['summaries'].append(data_train[ind]['summary'])

n_data_val = {'id': [], 'texts': [], 'summaries': []}

for ind in data_val.keys():
  n_data_val['id'].append(ind)
  n_data_val['texts'].append(data_val[ind]['text'])
  n_data_val['summaries'].append(data_val[ind]['summary'])

n_data_test = {'id': [], 'texts': [], 'summaries': []}

for ind in data_test.keys():
  n_data_test['id'].append(ind)
  n_data_test['texts'].append(data_test[ind]['text'])
  n_data_test['summaries'].append(data_test[ind]['summary'])

In [7]:
from datasets import Dataset

train_dataset = Dataset.from_dict(n_data_train)

In [8]:
train_dataset

Dataset({
    features: ['id', 'texts', 'summaries'],
    num_rows: 12925
})

In [9]:
val_dataset = Dataset.from_dict(n_data_val)
test_dataset = Dataset.from_dict(n_data_test)

In [10]:
max_input_length = 512
max_target_length = 120

def preprocess_function(examples):
    inputs = [doc for doc in examples["texts"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, 
                             truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summaries"],
                           max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

In [12]:
import gc

del data_train, data_val, data_test, n_data_train, n_data_test, n_data_val
del train_dataset, val_dataset
gc.collect() 

197

In [13]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [19]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True
)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
!pip install rouge_score

In [16]:
from datasets import load_metric

metric = load_metric("rouge")

In [17]:
import nltk
import numpy as np

nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [21]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.678800,1.518871,76.615300,0.625900,76.622300,76.670300,47.278700
2,1.427700,1.374009,80.521700,1.117800,80.515700,80.578100,47.631900
3,1.326900,1.313409,80.033600,1.556100,80.047600,80.079600,47.447400


KeyboardInterrupt: ignored

In [23]:
test_predictions = trainer.predict(tokenized_test_dataset)

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.678800,1.518871,76.615300,0.625900,76.622300,76.670300,47.278700
2,1.427700,1.374009,80.521700,1.117800,80.515700,80.578100,47.631900
3,1.326900,1.313409,80.033600,1.556100,80.047600,80.079600,47.447400


In [24]:
!pip install rouge

In [25]:
from rouge import Rouge

rouge = Rouge()

In [26]:
references = test_dataset['summaries']
predictions = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in test_predictions[0]]

scores = rouge.get_scores(predictions, references, avg=True)

In [27]:
scores

{'rouge-1': {'f': 0.3310628108175336,
  'p': 0.4464291154040704,
  'r': 0.27352381863680425},
 'rouge-2': {'f': 0.18253784581433558,
  'p': 0.24586907132374944,
  'r': 0.15116159974082696},
 'rouge-l': {'f': 0.33069004967221033,
  'p': 0.434232704952492,
  'r': 0.27557661041859305}}

In [37]:
import random

test_inds = random.sample(range(2684), 100)

In [29]:
test_dataset

Dataset({
    features: ['id', 'texts', 'summaries'],
    num_rows: 2684
})

In [38]:
for i in test_inds:
  print('------Original summary------:')
  print(test_dataset['summaries'][i])
  print('------Generated summary------:')
  print(predictions[i])
  print('\n')

------Original summary------:
Рассматривается дело o признании недействительной сделки по перечислению должником денежных средств и применении последствий недействительности сделки.  Требование удовлетворено, поскольку в результате совершения оспариваемой сделки причинен вред имущественным правам кредиторов путем безвозмездного вывода активов должника из конкурсной массы.
------Generated summary------:
заявляется требование o признании недеиствительными сделок по перечислению должником денежных средств в адрес должника. требование удовлетворено, поскольку на момент совершения оспариваемои сделки у должника имелись неисполненные обязательства перед кредиторами, в связи с чем должник


------Original summary------:
Заявляется требование o взыскании неустойки, штрафа.  Застройщиком нарушен срок передачи объекта долевого строительства. Право требования неустойки перешло к истцу на основании договора цессии.  Требование удовлетворено в части, поскольку размер неустойки снижен на основании с